# Protocol debugger

This notebook speeds up the process of debugging a protocol script.

---

Previously, the user would need to:
1. Manually save their protocol's notebook file as a Python script (`File` > `Download as...`)
1. ...and then (inside a python terminal), run: `opentrons_simulate my_protocol.py`

Note, if using custom labware defenitions, they would need to append `--custom-labware-path="C:\Custom Labware"`

---

The difficulty with the 'manual' method above is that:
- On each occasion the Python script is downloaded, the `.py` filename changed.
- The simulation log files were not easy to analyse in details.
- Defining the `--custom-labware-path` was easy to  
Essentially, the process is cumbersome which leads to users debugging their protocols too infrequently!

### Import the required libraries
Check to see if a module/package is already imported using `dir()`.
This is for efficiency when repeatedly running of this script.

In [169]:
if 'os' not in dir():
    import os

if 'simulate' not in dir():
    from opentrons.simulate import simulate

if 'io' not in dir():
    import io

if 'transform_notebook' not in dir():
    from notebooktoall.transform import transform_notebook

if 'pprint' not in dir():
    from pprint import pprint
    
if 'Tk' not in dir():
    from tkinter import Tk
    from tkinter.filedialog import askopenfilename, askdirectory

Note, manually installing `notebooktoall`, `tk` and `pprint` may be required.

If this is the case (and to guarantee the packages are installed for the current Python kernel / virtual environment):

1. Open either:
    - the Jupyter terminal
    - a cell in this workbook (and prefix the commands below with an `!` mark) 
1. ... and run `pip install notebooktoall` etc.

# User variables

### Notebook directory
Defining `notebook_path` using interactive dialogue box.

Note, the dialogue will only appear on the first run of this script. To redefine this variable, restart the kernel.

In [170]:
if 'notebook_path' not in dir():
    # withdraw tkinter root window
    root = Tk()
    root.attributes("-topmost", True) # raise to be in front of other windows
    root.withdraw()

    # open dialog box and return the path to the selected file
    notebook_path = askopenfilename(initialdir = os.getcwd(),
                                         title = "Select protocol's notebook",
                                         filetypes = (("Python notebooks","*.ipynb"),("all files","*.*"))) 

print('notebook_path:')
print(notebook_path)

notebook_path:
C:/Users/yannm/Opentrons/SampleProtocol_v5.0.0.ipynb


### Labware directory
Defining `custom_labware_directory` using interactive dialogue box.

Note, the dialogue will only appear on the first run of this script. To redefine this variable, restart the kernel.

In [171]:
# Custom labware
using_custom_labware = True # Set to True or False

if using_custom_labware:
    if 'custom_labware_directory' not in dir():
        # withdraw tkinter root window
        root = Tk()
        root.attributes("-topmost", True) # raise to be in front of other windows
        root.withdraw()

        # open dialog box and return the path to the entered folder
        custom_labware_directory = askdirectory(initialdir = os.getcwd(),
                                                title = "Enter the folder containing custom labware definitions") 
    
    # Only prints if using_custom_labware = True 
    print('custom_labware_directory:')
    print(custom_labware_directory)

custom_labware_directory:
C:/Users/yannm/Opentrons/Custom_labware


# Automated conversion

In [172]:
script_path = os.path.splitext(notebook_path)[0] + '.py'

if os.path.exists(script_path):
    print('Removing previous .py script...')
    os.remove(script_path)

Removing previous .py script...


### Perform the transformation

In [173]:
print('Generating new .py version...')

transform_notebook(ipynb_file=notebook_path,
                   export_list=['py'])

Generating new .py version...


In [174]:
#Check the conversion worked
if os.path.exists(script_path):
    print('File converted:')
    print(script_path)
else:
    raise ValueError('The <.py> file was not found')

File converted:
C:/Users/yannm/Opentrons/SampleProtocol_v5.0.0.py


# Protocol simulation

In [175]:
if using_custom_labware:
    with open(script_path, 'r') as file:
        run_log = simulate(protocol_file=file, custom_labware_paths = [custom_labware_directory])
else:
    with open(script_path, 'r') as file:
        run_log = simulate(protocol_file=file)

Volume mixing fraction at A1 of eppendorf on 2  is 0.33
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
Volume mixing fraction at A1 of eppendorf on 2  is 0.33
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
Immersion depth has been set >0, dispense will take place below the meniscus
{'constituents_number': 0, 'volume': 300.0, 'headroom': 30.0, 'vh_functions': {'v_given_h': <function gradations_to_vh

### Displaying logs...

In [176]:
# Set True/False as desired

if False: # pretty print of run_log (too much detail?)
    pprint(run_log)
    
if True: # print structured log (good detail)
    for entry in run_log[0]:
        print(entry['payload']['text'])
        try:
            print('\t', 'Location: ', entry['payload']['location'])
        except:
            print('\t No location in "payload" dictionary')
        try:
            print('\t', 'Instrument: ', entry['payload']['instrument'])
        except:
            print('\t No instrument in "payload" dictionary')
        print()

if False: # print basic log (standard GUI output)
    for entry in run_log[0]:
        print(entry['payload']['text'])

if False: # print unstructured log (ugly)
    print(run_log)

Picking up tip from A1 of tiprack on 5
	 Location:  A1 of tiprack on 5
	 Instrument:  P300 Single-Channel GEN1 on right mount

Aspirating 100.0 uL from A1 of falcon_50ml on 3 at 75.0 uL/sec
	 Location:  Location(point=Point(x=293.58, y=62.9, z=105.41108695652173), labware=A1 of falcon_50ml on 3)
	 Instrument:  P300 Single-Channel GEN1 on right mount

Dispensing 100.0 uL into A1 of falcon_50ml on 3 at 300.0 uL/sec
	 Location:  Location(point=Point(x=293.58, y=62.9, z=107.59999999999998), labware=A1 of falcon_50ml on 3)
	 Instrument:  P300 Single-Channel GEN1 on right mount

Aspirating 100.0 uL from A1 of falcon_50ml on 3 at 75.0 uL/sec
	 Location:  Location(point=Point(x=293.58, y=62.9, z=105.41108695652171), labware=A1 of falcon_50ml on 3)
	 Instrument:  P300 Single-Channel GEN1 on right mount

Dispensing 100.0 uL into A1 of falcon_50ml on 3 at 300.0 uL/sec
	 Location:  Location(point=Point(x=293.58, y=62.9, z=107.59999999999997), labware=A1 of falcon_50ml on 3)
	 Instrument:  P300 Sin